# Retail Store Sales — Data cleaning for business use

**Objetivo**  
Preparar un dataset de ventas retail para que pueda utilizarse de forma segura en Excel, sistemas CRM y reportes de negocio.

Este notebook documenta todo el proceso de limpieza de datos, con énfasis en:
- calidad de datos
- lógica de negocio
- decisiones trazables
- entrega final lista para Excel

In [1]:
## 1. Configuración y carga de datos

In [2]:
#importar librerias
import numpy as np 
import pandas as pd
import re 

In [3]:
# Carga de archivos 
file_path=r"C:\Users\andre\OneDrive\Data_Cleaning_Portfolio\Solucion_datos_contactos\Data_raw\retail_store_sales.csv"
df_raw=pd.read_csv(file_path)
df=df_raw.copy()
df_raw.head()

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


## 2. Auditoría

In [4]:
df.shape, df.columns.tolist()

((12575, 11),
 ['Transaction ID',
  'Customer ID',
  'Category',
  'Item',
  'Price Per Unit',
  'Quantity',
  'Total Spent',
  'Payment Method',
  'Location',
  'Transaction Date',
  'Discount Applied'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total Spent       11971 non-null  float64
 7   Payment Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction Date  12575 non-null  object 
 10  Discount Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB


In [6]:
# Tabla de % nulos
nulls = (
    df.isna()
      .mean()
      .mul(100)
      .round(2)
      .sort_values(ascending=False)
      .to_frame(name="null_pct")
)

nulls

,null_pct
Discount Applied,33.39
Item,9.65
Price Per Unit,4.84
Total Spent,4.80
Quantity,4.80
Transaction ID,0.00
Customer ID,0.00
Category,0.00
Payment Method,0.00
Location,0.00


In [7]:
# cantidad de categorías o valores disntintos por variable
cardinality = (
    df.nunique(dropna=False)  # 1️⃣ cuenta valores distintos (incluye NaN)
      .sort_values(ascending=False)  # 2️⃣ ordena de mayor a menor
      .to_frame(name="unique_values")
)

cardinality

,unique_values
Transaction ID,12575
Transaction Date,1114
Total Spent,228
Item,201
Price Per Unit,26
Customer ID,25
Quantity,11
Category,8
Payment Method,3
Discount Applied,3


In [8]:
# Duplicados completos (toda la fila)
full_duplicates = df.duplicated().sum()
full_duplicates

np.int64(0)

## 3. Calidad de datos
- Existencia de valores nulos en algunas columnas.
- Fechas almacenadas como texto.
- Cantidades numéricas almacenadas como valores decimales.
- Campos financieros que requieren validación de coherencia.
- Valores faltantes que no pueden recuperarse de forma confiable.

## 4. Corrección de tipos de datos

**Objetivo**
Asegurar que cada columna tenga el tipo de dato correcto para su uso.

**Acciones**
- Conversión de fechas al formato datetime.
- Conversión de cantidades a tipo entero (preservando nulos).
- Verificación de tipos numéricos en campos financieros.

**Por qué es importante**
Los tipos correctos permiten:
- filtrar y ordenar correctamente
- realizar agregaciones confiables
- evitar errores al trabajar en Excel

In [9]:
# Intentamos convertir transaction date a fecha
transaction_date_parsed = pd.to_datetime(
    df["Transaction Date"],   # columna original (texto)
    errors="coerce"           # si algo falla → NaT (no se rompe el código)
)

# Contamos cuántas fechas NO se pudieron convertir
invalid_dates = transaction_date_parsed.isna().sum()

invalid_dates

np.int64(0)

In [10]:
# datetime
df["Transaction Date"] = pd.to_datetime(df["Transaction Date"])

# Verificamos el tipo
df["Transaction Date"].dtype

dtype('<M8[ns]')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    12575 non-null  object        
 1   Customer ID       12575 non-null  object        
 2   Category          12575 non-null  object        
 3   Item              11362 non-null  object        
 4   Price Per Unit    11966 non-null  float64       
 5   Quantity          11971 non-null  float64       
 6   Total Spent       11971 non-null  float64       
 7   Payment Method    12575 non-null  object        
 8   Location          12575 non-null  object        
 9   Transaction Date  12575 non-null  datetime64[ns]
 10  Discount Applied  8376 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(7)
memory usage: 1.1+ MB


In [12]:
# Ver valores únicos (incluye nulos)
df["Discount Applied"].value_counts(dropna=False)

Discount Applied
True     4219
NaN      4199
False    4157
Name: count, dtype: int64

In [13]:
# Revisamos valores únicos de Quantity
sorted(df["Quantity"].dropna().unique())

[np.float64(1.0),
 np.float64(2.0),
 np.float64(3.0),
 np.float64(4.0),
 np.float64(5.0),
 np.float64(6.0),
 np.float64(7.0),
 np.float64(8.0),
 np.float64(9.0),
 np.float64(10.0)]

In [14]:
# Convertimos Quantity a entero
df["Quantity"] = df["Quantity"].astype("Int64")

# Verificamos
df["Quantity"].dtype, df["Quantity"].isna().sum()

(Int64Dtype(), np.int64(604))

## 5. Validación de Coherencia Financiera

In [15]:
# Calculamos total esperado
expected_total = df["Price Per Unit"] * df["Quantity"]

# Comparamos con Total Spent 
difference = (df["Total Spent"] - expected_total).abs()

# Filas inconsistentes 
inconsistent_mask = difference > 0.01

# Resumen
inconsistent_mask.sum()

np.int64(0)

In [16]:
# Estadísticas básicas de Price Per Unit
df["Price Per Unit"].describe()

count    11966.000000
mean        23.365912
std         10.743519
min          5.000000
25%         14.000000
50%         23.000000
75%         33.500000
max         41.000000
Name: Price Per Unit, dtype: float64

## 6. Estandarización de Datos Categóricos

In [17]:
# Valores únicos actuales de Category
df["Category"].value_counts()

Category
Electric household essentials         1591
Furniture                             1591
Food                                  1588
Milk Products                         1584
Butchers                              1568
Beverages                             1567
Computers and electric accessories    1558
Patisserie                            1528
Name: count, dtype: int64

In [18]:
# Limpieza básica de Category
df["Category"] = (
    df["Category"]
    .str.strip()        # quita espacios invisibles
    .str.title()        # formato consistente
)

# Verificamos que no cambió la cardinalidad
df["Category"].value_counts()

Category
Electric Household Essentials         1591
Furniture                             1591
Food                                  1588
Milk Products                         1584
Butchers                              1568
Beverages                             1567
Computers And Electric Accessories    1558
Patisserie                            1528
Name: count, dtype: int64

In [19]:
df["Payment Method"].value_counts()

Payment Method
Cash              4310
Digital Wallet    4144
Credit Card       4121
Name: count, dtype: int64

In [20]:
df["Payment Method"] = (
    df["Payment Method"]
    .str.strip()
    .str.title()
)

df["Payment Method"].value_counts()

Payment Method
Cash              4310
Digital Wallet    4144
Credit Card       4121
Name: count, dtype: int64

In [21]:
df["Location"].value_counts()

Location
Online      6354
In-store    6221
Name: count, dtype: int64

In [22]:
df["Location"] = (
    df["Location"]
    .str.strip()
    .str.title()
)

df["Location"].value_counts()

Location
Online      6354
In-Store    6221
Name: count, dtype: int64

## 7. Estrategia de manejo de valores nulos

**Enfoque**
Los valores nulos se gestionaron usando criterio de negocio, no suposiciones.

**Decisiones**
- Campos críticos de venta (Item, Price, Quantity, Total):  
  → Se eliminaron filas con valores faltantes.
- `Discount Applied`:  
  → Se conservaron los valores nulos porque representan información no registrada.

In [23]:
# Filas con Item nulo
mask_item_null = df["Item"].isna()

# Filas con nulos financieros
mask_financial_null = (
    df["Price Per Unit"].isna() |
    df["Quantity"].isna() |
    df["Total Spent"].isna()
)

# Resumen
mask_item_null.sum(), mask_financial_null.sum()

(np.int64(1213), np.int64(1213))

In [24]:
rows_before = df.shape[0]
rows_before

12575

In [25]:
df_clean = df.loc[~mask_item_null].copy()
df_clean.shape

(11362, 11)

In [26]:
df_clean.isna().sum()

Transaction ID         0
Customer ID            0
Category               0
Item                   0
Price Per Unit         0
Quantity               0
Total Spent            0
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    3783
dtype: int64

## 9. Dataset Final y Exportación

In [27]:
output_path = r"C:\Users\andre\OneDrive\Data_Cleaning_Portfolio\Solucion_datos_contactos\Excel_final\retail_store_sales_cleaned.xlsx"
df_clean.to_excel(output_path, index=False)